# MultiEval Example

This notebook demonstrates a basic parameter sweep with LensKits `MultiEval` class.

## Setup

We first need to import our libraries.

In [1]:
from lenskit.batch import MultiEval
from lenskit.crossfold import partition_users, SampleN
from lenskit.algorithms import basic, als
from lenskit.datasets import MovieLens
from lenskit import topn, util
import pandas as pd
import matplotlib.pyplot as plt

Progress bars are useful:

In [2]:
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()

C:\Users\michaelekstrand\Anaconda3\envs\lkpy-dev\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


It takes a little while to run things, and can get kinda quiet in here. Let's set up logging so we can see the logging output in the notebook's message stream:

In [3]:
util.log_to_notebook()

[   INFO] lenskit.util.log notebook logging configured


Then set up the data access.

In [4]:
mlsmall = MovieLens('../data/ml-latest-small')

## Experiment

We're going to run our evaluation and store its output in the `my-eval` directory, generating 20-item recommendation lists::

In [5]:
eval = MultiEval('my-eval', recommend=20)

We're going to use a 5-fold cross-validation setup.  We save the data into a list in memory so we have access to the test data later.  In a larger experiment, you might write the partitions to disk and pass the file names to `add_datasets`.

In [6]:
pairs = list(partition_users(mlsmall.ratings, 5, SampleN(5)))
eval.add_datasets(pairs, name='ML-Small')

[   INFO] lenskit.crossfold partitioning 100004 rows for 671 users into 5 partitions
[   INFO] lenskit.crossfold fold 0: selecting test ratings
[   INFO] lenskit.crossfold fold 0: partitioning training data
[   INFO] lenskit.crossfold fold 1: selecting test ratings
[   INFO] lenskit.crossfold fold 1: partitioning training data
[   INFO] lenskit.crossfold fold 2: selecting test ratings
[   INFO] lenskit.crossfold fold 2: partitioning training data
[   INFO] lenskit.crossfold fold 3: selecting test ratings
[   INFO] lenskit.crossfold fold 3: partitioning training data
[   INFO] lenskit.crossfold fold 4: selecting test ratings
[   INFO] lenskit.crossfold fold 4: partitioning training data


We're going to test explicit MF with several neighborhood sizes:

In [7]:
eval.add_algorithms([als.BiasedMF(f) for f in [20, 30, 40, 50]],
                    attrs=['features'], name='BiasedMF')

And implicit MF:

In [8]:
eval.add_algorithms([als.ImplicitMF(f) for f in [20, 30, 40, 50]],
                    attrs=['features'], name='ImplicitMF')

And add a popular baseline for comparison:

In [9]:
eval.add_algorithms(basic.Popular(), name='Pop')

And finally, we will run the experiment!

In [ ]:
eval.run(progress=tqdm)

[   INFO] lenskit.batch._multi starting run 1: als.BiasedMF(features=20, regularization=0.1) on ML-Small:1
[   INFO] lenskit.batch._multi adapting als.BiasedMF(features=20, regularization=0.1) into a recommender
[   INFO] lenskit.batch._multi training algorithm als.BiasedMF(features=20, regularization=0.1) on 99329 ratings
[   INFO] lenskit.algorithms.als [ 0ms] fitting bias model
[   INFO] lenskit.algorithms.basic building bias model for 99329 ratings
[   INFO] lenskit.algorithms.basic global mean: 3.543
[   INFO] lenskit.algorithms.basic computed means for 9056 items
[   INFO] lenskit.algorithms.basic computed means for 671 users
[   INFO] lenskit.algorithms.als [ 365ms] normalizing 671x9056 matrix (99329 nnz)
[   INFO] lenskit.algorithms.als [1.75s] training biased MF model with ALS for 20 features
[   INFO] lenskit.algorithms.als [5.03s] finished epoch 0 (|ΔP|=32.363, |ΔQ|=123.164)
[   INFO] lenskit.algorithms.als [5.04s] finished epoch 1 (|ΔP|=11.988, |ΔQ|=52.216)
[   INFO] lenski

[   INFO] lenskit.batch._recommend recommended for 135 users in 1.93s
[   INFO] lenskit.batch._multi generated recommendations in 2.15s
[   INFO] lenskit.batch._multi run 2: writing results to my-eval\recommendations.parquet
[   INFO] lenskit.batch._multi finished run 2: als.BiasedMF(features=30, regularization=0.1) on ML-Small:1
[   INFO] lenskit.batch._multi starting run 3: als.BiasedMF(features=40, regularization=0.1) on ML-Small:1
[   INFO] lenskit.batch._multi adapting als.BiasedMF(features=40, regularization=0.1) into a recommender
[   INFO] lenskit.batch._multi training algorithm als.BiasedMF(features=40, regularization=0.1) on 99329 ratings
[   INFO] lenskit.algorithms.als [ 0ms] fitting bias model
[   INFO] lenskit.algorithms.basic building bias model for 99329 ratings
[   INFO] lenskit.algorithms.basic global mean: 3.543
[   INFO] lenskit.algorithms.basic computed means for 9056 items
[   INFO] lenskit.algorithms.basic computed means for 671 users
[   INFO] lenskit.algorithms

[   INFO] binpickle.write pickled 1338 bytes with 10 buffers
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 4 workers
[   INFO] lenskit.batch._recommend recommending with TopN/als.BiasedMF(features=50, regularization=0.1) for 135 users (n_jobs=None)
[   INFO] lenskit.batch._recommend recommended for 135 users in 1.87s
[   INFO] lenskit.batch._multi generated recommendations in 2.07s
[   INFO] lenskit.batch._multi run 4: writing results to my-eval\recommendations.parquet
[   INFO] lenskit.batch._multi finished run 4: als.BiasedMF(features=50, regularization=0.1) on ML-Small:1
[   INFO] lenskit.batch._multi starting run 5: als.ImplicitMF(features=20, reg=0.1, w=40) on ML-Small:1
[   INFO] lenskit.batch._multi adapting als.ImplicitMF(features=20, reg=0.1, w=40) into a recommender
[   INFO] lenskit.batch._multi training algorithm als.ImplicitMF(features=20, reg=0.1, w=40) on 99329 ratings
[   INFO] lenskit.algorithms.als [ 14ms] training implicit MF model with ALS for 20

[   INFO] lenskit.batch._recommend recommended for 135 users in 2.04s
[   INFO] lenskit.batch._multi generated recommendations in 2.27s
[   INFO] lenskit.batch._multi run 6: writing results to my-eval\recommendations.parquet
[   INFO] lenskit.batch._multi finished run 6: als.ImplicitMF(features=30, reg=0.1, w=40) on ML-Small:1
[   INFO] lenskit.batch._multi starting run 7: als.ImplicitMF(features=40, reg=0.1, w=40) on ML-Small:1
[   INFO] lenskit.batch._multi adapting als.ImplicitMF(features=40, reg=0.1, w=40) into a recommender
[   INFO] lenskit.batch._multi training algorithm als.ImplicitMF(features=40, reg=0.1, w=40) on 99329 ratings
[   INFO] lenskit.algorithms.als [ 17ms] training implicit MF model with ALS for 40 features
[   INFO] lenskit.algorithms.als have 99329 observations for 671 users and 9056 items
[   INFO] lenskit.algorithms.als [ 72ms] finished epoch 0 (|ΔP|=4123.802, |ΔQ|=7.707)
[   INFO] lenskit.algorithms.als [ 127ms] finished epoch 1 (|ΔP|=227.793, |ΔQ|=2.915)
[   

[   INFO] lenskit.batch._multi finished run 8: als.ImplicitMF(features=50, reg=0.1, w=40) on ML-Small:1
[   INFO] lenskit.batch._multi starting run 9: Popular on ML-Small:1
[   INFO] lenskit.batch._multi training algorithm Popular on 99329 ratings
[   INFO] lenskit.algorithms.basic trained unrated candidate selector for 99329 ratings
[   INFO] lenskit.batch._multi trained algorithm Popular in  12ms
[   INFO] lenskit.batch._multi generating recommendations for 135 users for Popular
[   INFO] lenskit.sharing persisting Popular to C:\Users\MICHAE~1\AppData\Local\Temp\lkpy-72qe5_ky.bpk
[   INFO] binpickle.write pickled 967 bytes with 7 buffers
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 4 workers
[   INFO] lenskit.batch._recommend recommending with Popular for 135 users (n_jobs=None)
[   INFO] lenskit.batch._recommend recommended for 135 users in 1.21s
[   INFO] lenskit.batch._multi generated recommendations in 1.28s
[   INFO] lenskit.batch._multi run 9: writing resul

...

[   INFO] lenskit.algorithms.als [ 13ms] training implicit MF model with ALS for 30 features
[   INFO] lenskit.algorithms.als have 99334 observations for 671 users and 9051 items
[   INFO] lenskit.algorithms.als [ 66ms] finished epoch 0 (|ΔP|=3828.473, |ΔQ|=8.051)
[   INFO] lenskit.algorithms.als [ 117ms] finished epoch 1 (|ΔP|=203.809, |ΔQ|=3.198)
[   INFO] lenskit.algorithms.als [ 174ms] finished epoch 2 (|ΔP|=119.977, |ΔQ|=2.158)
[   INFO] lenskit.algorithms.als [ 230ms] finished epoch 3 (|ΔP|=124.732, |ΔQ|=1.720)
[   INFO] lenskit.algorithms.als [ 282ms] finished epoch 4 (|ΔP|=152.127, |ΔQ|=1.382)
[   INFO] lenskit.algorithms.als [ 337ms] finished epoch 5 (|ΔP|=124.034, |ΔQ|=1.110)
[   INFO] lenskit.algorithms.als [ 388ms] finished epoch 6 (|ΔP|=106.200, |ΔQ|=0.926)
[   INFO] lenskit.algorithms.als [ 440ms] finished epoch 7 (|ΔP|=99.108, |ΔQ|=0.813)
[   INFO] lenskit.algorithms.als [ 494ms] finished epoch 8 (|ΔP|=110.136, |ΔQ|=0.736)
[   INFO] lenskit.algorithms.als [ 548ms] finish

[   INFO] lenskit.algorithms.als [ 135ms] finished epoch 1 (|ΔP|=247.908, |ΔQ|=2.869)
[   INFO] lenskit.algorithms.als [ 199ms] finished epoch 2 (|ΔP|=226.543, |ΔQ|=1.851)
[   INFO] lenskit.algorithms.als [ 262ms] finished epoch 3 (|ΔP|=253.983, |ΔQ|=1.377)
[   INFO] lenskit.algorithms.als [ 331ms] finished epoch 4 (|ΔP|=205.471, |ΔQ|=1.045)
[   INFO] lenskit.algorithms.als [ 415ms] finished epoch 5 (|ΔP|=176.248, |ΔQ|=0.856)
[   INFO] lenskit.algorithms.als [ 486ms] finished epoch 6 (|ΔP|=158.412, |ΔQ|=0.735)
[   INFO] lenskit.algorithms.als [ 550ms] finished epoch 7 (|ΔP|=142.547, |ΔQ|=0.631)
[   INFO] lenskit.algorithms.als [ 610ms] finished epoch 8 (|ΔP|=147.098, |ΔQ|=0.582)
[   INFO] lenskit.algorithms.als [ 672ms] finished epoch 9 (|ΔP|=125.801, |ΔQ|=0.513)
[   INFO] lenskit.algorithms.als [ 730ms] finished epoch 10 (|ΔP|=127.546, |ΔQ|=0.484)
[   INFO] lenskit.algorithms.als [ 795ms] finished epoch 11 (|ΔP|=108.894, |ΔQ|=0.432)
[   INFO] lenskit.algorithms.als [ 853ms] finished e

[   INFO] lenskit.batch._recommend recommending with TopN/als.BiasedMF(features=20, regularization=0.1) for 134 users (n_jobs=None)
[   INFO] lenskit.batch._recommend recommended for 134 users in 1.86s
[   INFO] lenskit.batch._multi generated recommendations in 2.06s
[   INFO] lenskit.batch._multi run 37: writing results to my-eval\recommendations.parquet
[   INFO] lenskit.batch._multi finished run 37: als.BiasedMF(features=20, regularization=0.1) on ML-Small:5
[   INFO] lenskit.batch._multi starting run 38: als.BiasedMF(features=30, regularization=0.1) on ML-Small:5
[   INFO] lenskit.batch._multi adapting als.BiasedMF(features=30, regularization=0.1) into a recommender
[   INFO] lenskit.batch._multi training algorithm als.BiasedMF(features=30, regularization=0.1) on 99334 ratings
[   INFO] lenskit.algorithms.als [ 0ms] fitting bias model
[   INFO] lenskit.algorithms.basic building bias model for 99334 ratings
[   INFO] lenskit.algorithms.basic global mean: 3.543
[   INFO] lenskit.algo

[   INFO] lenskit.sharing persisting TopN/als.BiasedMF(features=40, regularization=0.1) to C:\Users\MICHAE~1\AppData\Local\Temp\lkpy-tx0mcmev.bpk
[   INFO] binpickle.write pickled 1339 bytes with 10 buffers
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 4 workers
[   INFO] lenskit.batch._recommend recommending with TopN/als.BiasedMF(features=40, regularization=0.1) for 134 users (n_jobs=None)
[   INFO] lenskit.batch._recommend recommended for 134 users in 2.00s
[   INFO] lenskit.batch._multi generated recommendations in 2.20s
[   INFO] lenskit.batch._multi run 39: writing results to my-eval\recommendations.parquet
[   INFO] lenskit.batch._multi finished run 39: als.BiasedMF(features=40, regularization=0.1) on ML-Small:5
[   INFO] lenskit.batch._multi starting run 40: als.BiasedMF(features=50, regularization=0.1) on ML-Small:5
[   INFO] lenskit.batch._multi adapting als.BiasedMF(features=50, regularization=0.1) into a recommender
[   INFO] lenskit.batch._multi trainin

[   INFO] lenskit.sharing persisting TopN/als.ImplicitMF(features=20, reg=0.1, w=40) to C:\Users\MICHAE~1\AppData\Local\Temp\lkpy-lo76n4d0.bpk
[   INFO] binpickle.write pickled 1194 bytes with 8 buffers
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 4 workers
[   INFO] lenskit.batch._recommend recommending with TopN/als.ImplicitMF(features=20, reg=0.1, w=40) for 134 users (n_jobs=None)
[   INFO] lenskit.batch._recommend recommended for 134 users in 1.80s
[   INFO] lenskit.batch._multi generated recommendations in 2.00s
[   INFO] lenskit.batch._multi run 41: writing results to my-eval\recommendations.parquet
[   INFO] lenskit.batch._multi finished run 41: als.ImplicitMF(features=20, reg=0.1, w=40) on ML-Small:5
[   INFO] lenskit.batch._multi starting run 42: als.ImplicitMF(features=30, reg=0.1, w=40) on ML-Small:5
[   INFO] lenskit.batch._multi adapting als.ImplicitMF(features=30, reg=0.1, w=40) into a recommender
[   INFO] lenskit.batch._multi training algorithm als.

## Analysis

Now that the experiment is run, we can read its outputs.

First the run metadata:

In [ ]:
runs = pd.read_csv('my-eval/runs.csv')
runs.set_index('RunId', inplace=True)
runs.head()

This describes each run - a data set, partition, and algorithm combination.  To evaluate, we need to get the actual recommendations, and combine them with this:

In [ ]:
recs = pd.read_parquet('my-eval/recommendations.parquet')
recs.head()

We're going to compute per-(run,user) evaluations of the recommendations *before* combining with metadata. 

In order to evaluate the recommendation list, we need to build a combined set of truth data. Since this is a disjoint partition of users over a single data set, we can just concatenate the individual test frames:

In [ ]:
truth = pd.concat((p.test for p in pairs), ignore_index=True)

Now we can set up an analysis and compute the results.

In [ ]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
raw_ndcg = rla.compute(recs, truth)
raw_ndcg.head()

Next, we need to combine this with our run data, so that we know what algorithms and configurations we are evaluating:

In [ ]:
ndcg = raw_ndcg.join(runs[['AlgoClass', 'features']], on='RunId')
ndcg.head()

We can compute the overall average performance for each algorithm configuration - fillna makes the group-by happy with Popular's lack of a feature count:

In [ ]:
ndcg.fillna(0).groupby(['AlgoClass', 'features'])['ndcg'].mean()

Now, we can plot this:

In [ ]:
mf_scores = ndcg.groupby(['AlgoClass', 'features'])['ndcg'].mean().reset_index()
pop_score = ndcg[ndcg['AlgoClass'] == 'Popular']['ndcg'].mean()
plt.axhline(pop_score, color='grey', linestyle='--', label='Popular')
for algo, data in mf_scores.groupby('AlgoClass'):
    plt.plot(data['features'], data['ndcg'], label=algo)
plt.legend()
plt.xlabel('Features')
plt.ylabel('nDCG')